# BLIŽINA, EKSCENTRIČNOST IN VMESNA CENTRALNOST

Pri najinem projektu bova s pomočjo treh različnih mer (ekscentričnosti, bližine in vmesne centralnosti) iskala in analizirala najpomembnejša vozlišča v grafih oziroma socialnih omrežjih. Vozlišče je pomembnejše, če ima višjo vrednost bližine in vmesne centralnosti ter nižjo vrednost ekscentričnosti. Natančnejši opisi pojmov so spodaj. Primerjala bova, kako pogosto je vozlišče, ki je pomembno z vidika ene mere, pomembno tudi v okviru ostalih dveh mer.

Najino glavno orodje za analizo grafov bo $Sage$, kjer bova generirala več grafov (približno 1000) v različnih velikostih (približno 10), grafe socialnih omrežij pa bova pridobila s spleta. Opazovala bova tudi, kako se vrednosti mer za najpomembnejša vozlišča spremenijo, če se omejimo na podgraf v določenem grafu. Grafi, ki jih bova pri projektu analizirala, bodo neusmerjeni.

## Bližina
Bližina je v povezanem grafu mera centralnosti, ki jo izračunamo kot
recipročno vsoto dolžin najkrajših poti med nekim vozliščem in vsemi
drugimi vozlišči v grafu. Bližje kot je opazovano vozlišče ostalim
vozliščem v grafu, bolj centralno je.

\begin{equation*}
C(x) = \frac{1}{\sum_{y}d(y,x)},
\end{equation*}

kjer je *d*(*y*, *x*) razdalja med vozliščema x in y. Pogosto se namesto
zgornje vrednosti izračuna povprečno dolžino najkrajše poti v grafu.
Dobimo jo tako, da zgornjo formulo pomnožimo z *N* − 1, kjer je *N*
število vseh vozlišč v grafu. Pri obsežnejših grafih se  − 1 izpusti iz
enačbe, zato se za bližino uporablja kar sledečo formulo:

\begin{equation*}
C(x) = \frac{N}{\sum_{y}d(y,x)}.
\end{equation*}

Pri usmerjenih grafih je potrebno upoštevati tudi smer povezav. Določeno
vozlišče ima lahko različno bližino za vhodne in izhodne povezave. V
nepovezanih grafih namesto recipročne vsote dolžin najkrajših poti med
vozlišči računamo vsoto recipročnih dolžin najkrajših poti med vozlišči.
Pri tem upoštevamo, da $1/\infty = 0$

\begin{equation*}
H(x) = \frac{N}{\sum_{y \neq x}d(y,x)}.
\end{equation*}

## Ekscentričnost (izsrednost)
Ekscentričnost nekega vozlišča $v$  v povezanem grafu $G$ označimo z $\epsilon(v)$ in je definirana kot maksimalna dolžina med vozliščem $v$ in katerimkoli drugim vozliščem v grafu $G$. V nepovezanih grafih imajo vsa vozlišča neskončno vrednost ekscentičnosti.
Maksimalno ekscentričnost v grafu imenujemo diameter (premer) grafa (najdaljša najkrajša pot med dvema vozliščema grafa), minimalno ekscentričnost pa polmer grafa.

## Vmesna centralnost

V teoriji grafov je vmesna centralnost mera centralizacije grafa, ki temelji na najkrajših poteh v grafu. Za vsak par vozlišč v povezanem grafu, obstaja vsaj ena najkrajša pot med vozliščema tako, da je katerokoli število povezav, po katerih gre ta pot (za neutežene grafe) ali pa vsota uteži na povezavah (za utežene grafe) minimalna. Vmesna centralnost za vsako vozlišče je število teh najkrajših poti, ki grejo skozi vozlišče. Vmesna centralnost se uporablja v mnogih problemih v teoriji omrežij, tudi v problemih povezanih s socialnimi omrežji, biologijo in transportom. V telekomunikacijskem omrežju ima vozlišče z višjo vrednostjo vmesne centralnosti večjo kontrolo nad omrežjem, ker bo več informacij teklo čez to vozlišče. Vmesna centralnost vozlišča $v$ je podana z izrazom:

\begin{equation*}
g(v) = \sum_{s \ne v \ne t}^{ } \frac{\sigma_{st} (v)}{\sigma_{st}}
\end{equation*}

Kjer je $\sigma_{st}$ skupno število najkrajših poti od vozlišča $s$ do vozlišča $t$ in $\sigma_{st} (v)$ je število teh poti, ki grejo skozi $v$. 

## Implementacija algoritma
Torej najprej se bova osredotočila na implementacijo algoritma, ki bo osnovan na podlagi Dijkstrovega algoritma, ki mu bova dodala vse tri mere, glede na katere bova ocenjevala grafe.

##### Dijkstrov algoritem

In [6]:
from collections import deque, namedtuple


# uporabimo neskončno za začetno razdaljo do vozlišč.
inf = float('inf')
#trojka z začetnm vozliščem, končnim vozliščem in ceno povezave
Edge = namedtuple('Edge', 'start, end, cost')


def make_edge(start, end, cost=1):
  return Edge(start, end, cost)


class Graph:
    def __init__(self, edges):
        # preverimo če so pravi podatki torej če lahko iz tega zgradimo graf
        wrong_edges = [i for i in edges if len(i) not in [2, 3]]
        if wrong_edges:
            raise ValueError('Wrong edges data: {}'.format(wrong_edges))

        self.edges = [make_edge(*edge) for edge in edges]
        vmesna = self.edges
        nova = []
        for edge in vmesna:
            nova.append(make_edge(edge.end, edge.start))
        self.edges = vmesna + nova
        

    @property
    def vertices(self):
        #dobimo množico vozlišč, ki so v grafu
        return set(
            sum(
                ([edge.start, edge.end] for edge in self.edges), []
            )
        )

    def get_node_pairs(self, n1, n2, both_ends=True):
        #dobimo pare vozlišč
        if both_ends:
            node_pairs = [[n1, n2], [n2, n1]]
        else:
            node_pairs = [[n1, n2]]
        return node_pairs

    def remove_edge(self, n1, n2, both_ends=True):
        #funkcija, ki odstrani povezavo med vozliščema
        node_pairs = self.get_node_pairs(n1, n2, both_ends)
        edges = self.edges[:]
        for edge in edges:
            if [edge.start, edge.end] in node_pairs:
                self.edges.remove(edge)

    def add_edge(self, n1, n2, cost=1, both_ends=True):
        #funkcija, ki doda povezavo med vozliščema
        node_pairs = self.get_node_pairs(n1, n2, both_ends)
        for edge in self.edges:
            if [edge.start, edge.end] in node_pairs:
                return ValueError('Edge {} {} already exists'.format(n1, n2))

        self.edges.append(Edge(start=n1, end=n2, cost=cost))
        if both_ends:
            self.edges.append(Edge(start=n2, end=n1, cost=cost))

    @property
    def neighbours(self):
        #funkcija, ki najde sosede vsakega vozlišča
        neighbours = {vertex: set() for vertex in self.vertices}
        for edge in self.edges:
            neighbours[edge.start].add((edge.end, edge.cost))

        return neighbours

    def dijkstra(self, source, dest):
        assert source in self.vertices, 'Such source node doesn\'t exist'
        #nastavimo vse razdalje na začetku na neskončno
        distances = {vertex: inf for vertex in self.vertices}
        #prejšna vozlišča, nazačetki vse na None
        previous_vertices = {
            vertex: None for vertex in self.vertices
        }
        #razdalja od začetka do začetka je 0
        distances[source] = 0
        #naredimo kopijo
        vertices = self.vertices.copy()
        #dokler imamo za preverit še kakšno vozlišle
        while vertices:
            #trenutno vozlišče
            current_vertex = min(
                vertices, key=lambda vertex: distances[vertex])
            vertices.remove(current_vertex)
            if distances[current_vertex] == inf:
                break
            for neighbour, cost in self.neighbours[current_vertex]:
                alternative_route = distances[current_vertex] + cost
                if alternative_route < distances[neighbour]:
                    distances[neighbour] = alternative_route
                    previous_vertices[neighbour] = current_vertex

        path, current_vertex = deque(), dest
        while previous_vertices[current_vertex] is not None:
            path.appendleft(current_vertex)
            current_vertex = previous_vertices[current_vertex]
        if path:
            path.appendleft(current_vertex)
        return path


graph = Graph([
    ("a", "b"),("a", "f"), ("c", "d"), ("c", "f"),  ("d", "e"), ("g", "g"),
    ("e", "f")])

graph.dijkstra("a", "e")


deque(['a', 'f', 'e'])

##### Bližina

In [12]:
def blizina(graph):
    seznam=[]
    for x in graph.vertices:
        D = 0
        for y in graph.vertices:
            D = D + (len(graph.dijkstra(x, y)))
        if D == 0:
            C = float('inf')
        else:
            C = 1/D
        seznam.append((x,C))
    print(seznam)
blizina(graph)

[('a', 1/14), ('c', 1/14), ('b', 1/18), ('e', 1/14), ('d', 1/16), ('g', inf), ('f', 1/12)]


##### Ekscentričnost (Izsrednost)

In [14]:
def ekscentricnost(graph):
    seznam=[]
    for x in graph.vertices:
        D = 0
        for y in graph.vertices:
            print(x, y, D, len(graph.dijkstra(x, y)))
            D = max(D, len(graph.dijkstra(x, y)))
        if D == 0:
            D = float('inf')
        seznam.append((x,D))
    print(seznam)
ekscentricnost(graph)

('a', 'a', 0, 0)
('a', 'c', 0, 3)
('a', 'b', 3, 2)
('a', 'e', 3, 3)
('a', 'd', 3, 4)
('a', 'g', 4, 0)
('a', 'f', 4, 2)
('c', 'a', 0, 3)
('c', 'c', 3, 0)
('c', 'b', 3, 4)
('c', 'e', 4, 3)
('c', 'd', 4, 2)
('c', 'g', 4, 0)
('c', 'f', 4, 2)
('b', 'a', 0, 2)
('b', 'c', 2, 4)
('b', 'b', 4, 0)
('b', 'e', 4, 4)
('b', 'd', 4, 5)
('b', 'g', 5, 0)
('b', 'f', 5, 3)
('e', 'a', 0, 3)
('e', 'c', 3, 3)
('e', 'b', 3, 4)
('e', 'e', 4, 0)
('e', 'd', 4, 2)
('e', 'g', 4, 0)
('e', 'f', 4, 2)
('d', 'a', 0, 4)
('d', 'c', 4, 2)
('d', 'b', 4, 5)
('d', 'e', 5, 2)
('d', 'd', 5, 0)
('d', 'g', 5, 0)
('d', 'f', 5, 3)
('g', 'a', 0, 0)
('g', 'c', 0, 0)
('g', 'b', 0, 0)
('g', 'e', 0, 0)
('g', 'd', 0, 0)
('g', 'g', 0, 0)
('g', 'f', 0, 0)
('f', 'a', 0, 2)
('f', 'c', 2, 2)
('f', 'b', 2, 3)
('f', 'e', 3, 2)
('f', 'd', 3, 3)
('f', 'g', 3, 0)
('f', 'f', 3, 0)
[('a', 4), ('c', 4), ('b', 5), ('e', 4), ('d', 5), ('g', inf), ('f', 3)]
